# Koopman Training and Validation for 2D Tail-Actuated Robotic Fish

This file uses experimental measurements using a 2D Tail-Actuated Robotic Fish to train an approximate Koopman operator. Using the initial conditions of each experiment, the data-driven solution is then used to predict the system forward and compared against the real experimental measurements. All fitness plots are generated at the end.

## Import Data

In [0]:
%%capture # suppress cell output
!git clone https://github.com/giorgosmamakoukas/DataSet.git # Import data from user location
!mv DataSet/* ./ # Move 'DataSet' folder to main directory

## Import User Functions

In [0]:
# This file includes all user-defined functions

from math import atan, sqrt, sin, cos
from numpy import empty, sign, dot,zeros
from scipy import io, linalg

def Psi_k(s,u): #Creates a vector of basis functions using states s and control u
    x, y, psi, v_x, v_y, omega = s # Store states in local variables

    if (v_y == 0) and (v_x == 0):
        atanvXvY = 0; # 0/0 gives NaN
        psi37 = 0;
        psi40 = 0;
        psi52 = 0;
        psi56 = 0;
    else:
        atanvXvY = atan(v_y/v_x);
        psi37 = v_x * pow(v_y,2) * omega/sqrt(pow(v_x,2)+pow(v_y,2));
        psi40 = pow(v_x,2) * v_y * omega / sqrt(pow(v_x,2) + pow(v_y,2)) * atanvXvY;
        psi52 = pow(v_x,2) * v_y * omega / sqrt(pow(v_x,2) + pow(v_y,2));
        psi56 = v_x * pow(v_y,2) * omega * atanvXvY / sqrt(pow(v_x,2) + pow(v_y,2));

    Psi = empty([62,1]); # declare memory to store psi vector

    # System States
    Psi[0,0] = x;
    Psi[1,0] = y;
    Psi[2,0] = psi;
    Psi[3,0] = v_x;
    Psi[4,0] = v_y;
    Psi[5,0] = omega;

    # f(t): terms that appear in dynamics
    Psi[6,0] = v_x * cos(psi) - v_y * sin(psi);
    Psi[7,0] = v_x * sin(psi) + v_y * cos(psi);
    Psi[8,0] =  v_y * omega;
    Psi[9,0] = pow(v_x,2);
    Psi[10,0] = pow(v_y,2);
    Psi[11,0] = v_x * omega;
    Psi[12,0] = v_x * v_y;
    Psi[13,0] = sign(omega) * pow(omega,2);

    # df(t)/dt: terms that appear in derivative of dynamics
    Psi[14,0] = v_y * omega * cos(psi);
    Psi[15,0] = pow(v_x,2) * cos(psi);
    Psi[16,0] = pow(v_y,2) * cos(psi);
    Psi[17,0] = v_x * omega * sin(psi);
    Psi[18,0] = v_x * v_y * sin(psi);

    Psi[19,0] = v_y * omega * sin(psi);
    Psi[20,0] = pow(v_x,2) * sin(psi);
    Psi[21,0] = pow(v_y,2) * sin(psi);
    Psi[22,0] = v_x * omega * cos(psi);
    Psi[23,0] = v_x * v_y * cos(psi);

    Psi[24,0] = v_x * pow(omega,2);
    Psi[25,0] = v_x * v_y * omega;
    Psi[26,0] = v_x * pow(v_y,2);
    Psi[27,0] = v_y * sign(omega) * pow(omega,2);
    Psi[28,0] = pow(v_x,3);

    Psi[29,0] = v_y * pow(omega,2);
    Psi[30,0] = v_x * omega * sqrt(pow(v_x,2) + pow(v_y,2));
    Psi[31,0] = v_y * omega * sqrt(pow(v_x,2) + pow(v_y,2)) * atanvXvY;
    Psi[32,0] = pow(v_x,2) * v_y;
    Psi[33,0] = v_x * sign(omega) * pow(omega,2);
    Psi[34,0] = pow(v_y,3);
    Psi[35,0] = pow(v_x,3) * atanvXvY;
    Psi[36,0] = v_x * pow(v_y,2) * atanvXvY;
    Psi[37,0] = psi37;
    Psi[38,0] = pow(v_x,2) * v_y * pow(atanvXvY,2);
    Psi[39,0] = pow(v_y,3) * pow(atanvXvY,2);
    Psi[40,0] = psi40;

    Psi[41,0] = pow(v_y,2) * omega;
    Psi[42,0] = v_x * v_y * sqrt(pow(v_x,2) + pow(v_y,2));
    Psi[43,0] = pow(v_y,2) * sqrt(pow(v_x,2) + pow(v_y,2)) * atanvXvY;
    Psi[44,0] = pow(v_x,2) * omega;
    Psi[45,0] = pow(v_x,2) * sqrt(pow(v_x,2) + pow(v_y,2)) * atanvXvY;
    Psi[46,0] = v_x * v_y * sign(omega) * omega;
    Psi[47,0] = pow(omega, 3);

    Psi[48,0] = v_y * omega * sqrt(pow(v_x,2) + pow(v_y,2));
    Psi[49,0] = pow(v_x,3);
    Psi[50,0] = v_x * pow(v_y,2);
    Psi[51,0] = pow(v_x,2) * v_y * atanvXvY;
    Psi[52,0] = psi52;

    Psi[53,0] = v_x * omega * sqrt(pow(v_x,2) + pow(v_y,2)) * atanvXvY;
    Psi[54,0] = pow(v_x,3) * pow(atanvXvY,2);
    Psi[55,0] = v_x * pow(v_y,2) * pow(atanvXvY,2);
    Psi[56,0] = psi56;
    Psi[57,0] = pow(v_y, 3) * atanvXvY;

    Psi[58,0] = v_x * pow(omega,2);
    Psi[59,0] = v_y * sign(omega) * pow(omega,2);

    # add control inputs
    Psi[60,0] = u[0];
    Psi[61,0] = u[1];

    return Psi

def A_and_G(s_1, s_2, u): # Uses measurements s(t_k) & s(t_{k+1}) to calculate A and G 
    A = dot(Psi_k(s_2, u), Psi_k(s_1, u).transpose());
    G = dot(Psi_k(s_1, u), Psi_k(s_1, u).transpose());
    return A, G

def TrainKoopman(): # Train an approximate Koopman operator

  ######## 1. IMPORT DATA ########

  mat = io.loadmat('InterpolatedData_200Hz.mat', squeeze_me=True)
  positions =  mat['Lengths'] - 1 # subtract 1 to convert MATLAB indices to python
  x = mat['x_int_list']
  y = mat['y_int_list']
  psi = mat['psi_int_list']
  v_x = mat['v1_int_list']
  v_y = mat['v2_int_list']
  omega = mat['omega_int_list']
  u1 = mat['u1_list']
  u2 = mat['u2_list']

  ######## 2. INITIALIZE A and G matrices
  A = zeros((62, 62)) # 62 is the size of the Ψ basis functions
  G = zeros((62, 62))

  ######## 3. TRAINING KOOPMAN ########
  for i in range(x.size-1):

      # print('{:.2f} % completed'.format(i/x.size*100))
      if i in positions:
          i += 1 # jump to next trial at the end of each trial

      # Create pair of state measurements
      s0 = [x[i], y[i], psi[i], v_x[i], v_y[i], omega[i]]
      sn = [x[i+1], y[i+1], psi[i+1], v_x[i+1], v_y[i+1], omega[i+1]]

      Atemp, Gtemp = A_and_G(s0,sn,[u1[i],u2[i]])
      A = A+Atemp;
      G = G+Gtemp;

  Koopman_d = dot(A,linalg.pinv2(G)) # more accurate than numpy
  # Koopman_d = dot(A,numpy.linalg.pinv(G))

  # io.savemat('SavedData.mat', {'A' : A, 'G': G, 'Kd': Koopman_d}) # save variables to Matlab file

  return Koopman_d

## Train Koopman & Test Fitness 

In [16]:
# This file trains and tests the accuracy of approximate Koopman operator

######## 0. IMPORT PYTHON FUNCTIONS ########

import matplotlib.pyplot as plt
from numpy import arange, insert, linspace

######## 1. IMPORT EXPERIMENTAL DATA ########

mat = io.loadmat('InterpolatedData_200Hz.mat', squeeze_me=True)
positions =  mat['Lengths'] - 1 # subtract 1 to convert MATLAB indices to python
# positions includes indices with last measurement of each experiment
x = mat['x_int_list']
y = mat['y_int_list']
psi = mat['psi_int_list']
v_x = mat['v1_int_list']
v_y = mat['v2_int_list']
omega = mat['omega_int_list']
u1 = mat['u1_list']
u2 = mat['u2_list']
positions = insert(positions, 0, -1) # insert -1 as index that precedes the 1st experiment

######## 2. PREDICT DATA USING TRAINED KOOPMAN ########

Kd = TrainKoopman() # Train Koopman
for exp_i in range(0, positions.size -2): # for each experiment
    indx = positions[exp_i]+1 # beginning index of each trial

    Psi_predicted = empty((positions[exp_i+1]-(indx), 62))
    s0 = [x[indx], y[indx], psi[indx], v_x[indx], v_y[indx], omega[indx]]
    Psi_predicted[0,:] = Psi_k(s0, [u1[indx], u2[indx]]).transpose() # Initialize with same initial conditions as experiment

    for j in range(0, positions[exp_i+1]-1-(indx)):
        Psi_predicted[j+1, :] = dot(Kd,Psi_predicted[j, :])

    ######## 3. PLOT EXPERIMENTAL VS PREDICTED DATA ########

    ylabels = ['x (m)', 'y (m)', 'ψ (rad)', r'$\mathregular{v_x (m/s)}$', r'$\mathregular{v_x (m/s)}$', 'ω (rad/s)']
    exp_data = [x, y, psi, v_x, v_y, omega]

    time = linspace(0, 1./200*(j+2), j+2) # create time vector
    fig = plt.figure()
    for states_i in range(6):
        plt.subplot('23'+str(states_i+1)) # 2 rows # 3 columns
        plt.plot(time, Psi_predicted[:, states_i])
        plt.plot(time, exp_data[states_i][indx:positions[exp_i+1]])
        plt.ylabel(ylabels[states_i])
    plt.gca().legend(('Predicted','Experimental'))

    Amp_values = [15, 20, 25, 30]
    Bias_values = [-20, -30, -40, -50, 0, 20, 30, 40, 50]
    titles = 'Amp: ' + str(Amp_values[(exp_i)//18]) + ' Bias: ' + str(Bias_values[(exp_i % 18) //2])
    fig.suptitle(titles)
    plt.show(block=False)


Output hidden; open in https://colab.research.google.com to view.